[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wp-Zhang/HandyRec/blob/master/examples/YouTubeDNN.ipynb)

> This notebook runs YouTubeDNN including candidate generation and ranking on MovieLens1M dataset. 

> Only movies with ratings larger than 3 are treated as 'positive' samples for each user. Every last 10 'positive' movies of each user are held out for testing.

## Table of Contents:
* [Prepare data for matching](#section-0)
* [Train match model and export embeddings](#section-1)
* [Use Faiss to generate candidates](#section-2)
* [Train rank model and predict](#section-3)

**Download dataset and install packages**

In [ ]:
! git clone https://github.com/Wp-Zhang/HandyRec.git
! pip install faiss-cpu

Cloning into 'HandyRec'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 226 (delta 93), reused 139 (delta 31), pack-reused 0
Receiving objects: 100% (226/226), 247.54 KiB | 10.76 MiB/s, done.
Resolving deltas: 100% (93/93), done.
     |████████████████████████████████| 8.6 MB 8.8 MB/s 


In [ ]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! unzip -o ml-1m.zip

--2022-03-16 05:57:47--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  13.4MB/s    in 0.4s    

2022-03-16 05:57:48 (13.4 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


**Import relative packages**

In [ ]:
import sys
sys.path.append('./HandyRec/')

In [ ]:
from handyrec.dataset.movielens import MovieMatchDataHelper, MovieRankDataHelper
from handyrec.layers.utils import sampledsoftmaxloss
from handyrec.models.match import YouTubeMatchDNN
from handyrec.models.rank import YouTubeRankDNN
from handyrec.features import DenseFeature, SparseFeature, SparseSeqFeature
from handyrec.dataset.metrics import mapk, recall_at_k
from handyrec.models.utils import search_embedding

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
import numpy as np
import pandas as pd
import gc

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
MATCH_EMBEDDING_DIM = 256
RANK_EMBEDDING_DIM = 256
SEQ_LEN = 40
BATCH_SIZE = 2**12
NEPOCH = 50

NEG_NUM = 10
CANDIDATE_NUM = 100

# 0. Prepare data for matching<a name="section-0"></a>

In [ ]:
match_dh = MovieMatchDataHelper('./ml-1m/')
match_user_features = ['user_id','gender','age','occupation', 'zip']
match_movie_features = ['movie_id']

data = match_dh.get_clean_data(sparse_features=['gender','age','occupation', 'zip'])
match_dh.gen_data_set(match_user_features+match_movie_features, data, seq_max_len=SEQ_LEN)

Encode User Sparse Feats: 100%|██████████| 4/4 [00:00<00:00, 166.04it/s]
Encode Item Sparse Feats: 0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00,  4.19it/s]
0it [00:00, ?it/s]


In [ ]:
match_train, match_train_label, match_test, match_test_label = match_dh.load_dataset(match_user_features, match_movie_features)

Load movie Features: 100%|██████████| 1/1 [00:00<00:00, 306.87it/s]


In [ ]:
match_feature_dim = match_dh.get_feature_dim(data, match_user_features, match_movie_features, [])

# 1. Train match model and export embeddings <a name="section-1"></a>

In [ ]:
# * add example_age^2 as showed in the original paper
match_train['example_age_2'] = match_train['example_age']**2
match_test['example_age_2'] = match_test['example_age']**2

In [ ]:
match_user_dense_feats = ['example_age','example_age_2']
match_user_sparse_feats = ['user_id','gender','age','occupation', 'zip']

In [ ]:
match_user_features = [SparseFeature(x, match_feature_dim[x], MATCH_EMBEDDING_DIM) for x in match_user_sparse_feats] +\
                [DenseFeature(x) for x in match_user_dense_feats] +\
                [SparseSeqFeature(SparseFeature('movie_id', match_feature_dim['movie_id'], MATCH_EMBEDDING_DIM), 'hist_movie_id',SEQ_LEN)]
match_item_id = SparseFeature('movie_id', match_feature_dim['movie_id'], MATCH_EMBEDDING_DIM)

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
match_model = YouTubeMatchDNN(
    match_user_features, match_item_id, num_sampled=100, 
    user_dnn_hidden_units=(1024,512,MATCH_EMBEDDING_DIM), dnn_dropout=0.2
)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
match_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3), loss=sampledsoftmaxloss)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./match_checkpoint/',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
history = match_model.fit(match_train, match_train_label,
                            batch_size=BATCH_SIZE, 
                            epochs=NEPOCH,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[early_stop,checkpoint])
# model.save_weights('youtubematch.h5')

Train on 845828 samples, validate on 93981 samples
Epoch 1/50
845828/845828 [==============================] - 24s 29us/sample - loss: 3.7388 - val_loss: 3.6169
Epoch 2/50
845828/845828 [==============================] - 16s 19us/sample - loss: 3.1036 - val_loss: 3.2072
Epoch 3/50
845828/845828 [==============================] - 16s 19us/sample - loss: 2.7556 - val_loss: 2.8066
Epoch 4/50
845828/845828 [==============================] - 16s 19us/sample - loss: 2.5263 - val_loss: 2.4278
Epoch 5/50
845828/845828 [==============================] - 17s 20us/sample - loss: 2.4017 - val_loss: 2.3297
Epoch 6/50
845828/845828 [==============================] - 16s 19us/sample - loss: 2.2508 - val_loss: 2.2447
Epoch 7/50
845828/845828 [==============================] - 17s 20us/sample - loss: 2.1634 - val_loss: 2.2130
Epoch 8/50
845828/845828 [==============================] - 16s 19us/sample - loss: 2.0873 - val_loss: 2.1024
Epoch 9/50
845828/845828 [==============================] - 16s 19us/

In [ ]:
# model.load_weights('youtubematch.h5')
match_model.load_weights('./match_checkpoint/')

In [ ]:
all_item_model_input = {"movie_id": data['item']['movie_id'].values}

user_embedding_model = Model(inputs=match_model.user_input, outputs=match_model.user_embedding)
item_embedding_model = Model(inputs=match_model.item_input, outputs=match_model.item_embedding)

user_embs = user_embedding_model.predict(match_test, batch_size=2 ** 15)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 15)

print(user_embs.shape)
print(item_embs.shape)

(6040, 256)
(3883, 256)


# 2. Use Faiss to generate candidates <a name="section-2"></a>

## Test match model

In [ ]:
candidates = search_embedding(
    MATCH_EMBEDDING_DIM, 
    item_embs, 
    user_embs,
    data['item']['movie_id'].values,
    CANDIDATE_NUM)

In [ ]:
mapk(match_test_label, candidates, k=10)

0.03744859665720592

In [ ]:
recall_at_k(match_test_label, candidates, k=10)

0.08903973509933776

In [ ]:
recall_at_k(match_test_label, candidates, k=100)

0.47832781456953644

## Prepare data for ranking

In [ ]:
test_user_embs = user_embedding_model.predict(match_test, batch_size=2 ** 15)
test_candidates = search_embedding(
    MATCH_EMBEDDING_DIM, 
    item_embs, 
    test_user_embs,
    data['item']['movie_id'].values,
    CANDIDATE_NUM)

test_candidates = {
    match_test['user_id'][i] : test_candidates[i]
    for i in range(test_candidates.shape[0])
}

In [ ]:
del user_embs, item_embs, match_train, match_train_label, test_user_embs
gc.collect()

50

In [ ]:
rank_dh = MovieRankDataHelper('./ml-1m/')
rank_user_features = ['user_id','gender','age','occupation', 'zip']
rank_movie_features = [f for f in data['item'].columns if f != 'title']

rank_dh.gen_data_set(rank_user_features+rank_movie_features, data, test_candidates, seq_max_len=SEQ_LEN, negnum=NEG_NUM)

100%|██████████| 18/18 [00:08<00:00,  2.18it/s]


In [ ]:
rank_train, rank_train_label, rank_test = rank_dh.load_dataset(rank_user_features, rank_movie_features)

Load movie Features: 100%|██████████| 19/19 [00:06<00:00,  2.87it/s]


In [ ]:
rank_train['example_age_2'] = rank_train['example_age']**2
rank_test['example_age_2'] = rank_test['example_age']**2
rank_train['time_gap_2'] = rank_train['time_gap']**2
rank_train['time_gap_square'] = np.sqrt(rank_train['time_gap'])
rank_test['time_gap_2'] = rank_test['time_gap']**2
rank_test['time_gap_square'] = np.sqrt(rank_test['time_gap'])

In [ ]:
rank_feature_dim = rank_dh.get_feature_dim(data, rank_user_features, rank_movie_features, [])

# 3. Train rank model and predict <a name="section-3"></a>

In [ ]:
rank_user_dense_feats = ['time_gap','time_gap_2','time_gap_square','example_age','example_age_2']
rank_user_sparse_feats = ['user_id','gender','age','occupation', 'zip']
rank_item_dense_feats = [f for f in rank_movie_features if f !='movie_id']
rank_item_sparse_feats = ['movie_id']

In [ ]:
# * change the improper feature name so it can be the name of a tf component
for i, k in enumerate(rank_item_dense_feats):
    if k == "Children's":
        rank_item_dense_feats[i] = 'Children'
rank_train['Children'] = rank_train.pop("Children's")
rank_test['Children'] = rank_test.pop("Children's")

In [ ]:
rank_user_features = [SparseFeature(x, rank_feature_dim[x], RANK_EMBEDDING_DIM) for x in rank_user_sparse_feats] +\
                [DenseFeature(x) for x in rank_user_dense_feats] +\
                [SparseSeqFeature(SparseFeature('movie_id', rank_feature_dim['movie_id'], RANK_EMBEDDING_DIM), 'hist_movie_id',SEQ_LEN)]
rank_item_feats = [SparseFeature(x, rank_feature_dim[x], RANK_EMBEDDING_DIM) for x in rank_item_sparse_feats] +\
                [DenseFeature(x) for x in rank_item_dense_feats]

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
rank_model = YouTubeRankDNN(
    rank_user_features, rank_item_feats, 
    dnn_hidden_units=(512,RANK_EMBEDDING_DIM), dnn_dropout=0.2
)

In [ ]:
rank_model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3), loss=binary_crossentropy)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
# checkpoint = tf.keras.callbacks.ModelCheckpoint(
#     filepath='./rank_checkpoint/',
#     save_weights_only=True,
#     monitor='val_loss',
#     mode='min',
#     save_best_only=True)
history = rank_model.fit(rank_train, rank_train_label,
                    batch_size=BATCH_SIZE*8, 
                    epochs=50,
                    verbose=1,
                    validation_split=0.15,
                    callbacks=[early_stop])
rank_model.save_weights('youtuberank.h5')

Train on 8787214 samples, validate on 1550685 samples
Epoch 1/50
8787214/8787214 [==============================] - 143s 16us/sample - loss: 3.1168 - val_loss: 1.4307
Epoch 2/50
8787214/8787214 [==============================] - 140s 16us/sample - loss: 1.9622 - val_loss: 1.6708
Epoch 3/50
8787214/8787214 [==============================] - 140s 16us/sample - loss: 1.6182 - val_loss: 1.5836
Epoch 4/50
8787214/8787214 [==============================] - 140s 16us/sample - loss: 1.3860 - val_loss: 1.2270
Epoch 5/50
8787214/8787214 [==============================] - 140s 16us/sample - loss: 1.1525 - val_loss: 0.9817
Epoch 6/50
8787214/8787214 [==============================] - 140s 16us/sample - loss: 0.9304 - val_loss: 0.7508
Epoch 7/50
8787214/8787214 [==============================] - 140s 16us/sample - loss: 0.8319 - val_loss: 0.6486
Epoch 8/50
8787214/8787214 [==============================] - 140s 16us/sample - loss: 0.7926 - val_loss: 0.6210
Epoch 9/50
8787214/8787214 [==============

In [ ]:
rank_model.load_weights('youtuberank.h5')

In [ ]:
del rank_train
gc.collect()

54

In [ ]:
pred = rank_model.predict(rank_test)

In [ ]:
pred_df = pd.DataFrame(columns=['user_id','movie_id','pred'])
pred_df['user_id'] = rank_test['user_id']
pred_df['movie_id'] = rank_test['movie_id']
pred_df['pred'] = pred

pred_df = pred_df.sort_values(by=['user_id','pred'], ascending=False).reset_index(drop=True)
pred_df = pred_df.groupby('user_id')['movie_id'].apply(list).reset_index()

In [ ]:
test_label_df = pd.DataFrame(columns=['user_id','label'])
test_label_df['user_id'] = match_test['user_id']
test_label_df['label'] = match_test_label.tolist()

In [ ]:
test_label_df = pd.merge(test_label_df, pred_df, on=['user_id'], how='left')

In [ ]:
mapk(test_label_df['label'], test_label_df['movie_id'], k=10)

0.029301672711027016

In [ ]:
recall_at_k(test_label_df['label'], test_label_df['movie_id'], k=10)

0.07064569536423841

In [ ]:
recall_at_k(test_label_df['label'], test_label_df['movie_id'], k=100)

0.47832781456953644